# Lab 5: Advanced CLI Operations for Insurance Monitoring
## Python Alternative Implementation
### Duration: 45 minutes

This lab covers advanced Redis monitoring techniques for insurance systems using Python.

## Setup and Initial Connection

First, let's import the required libraries and establish a connection to Redis.

In [ ]:
# Import required libraries
import redis
import time
import json
import sys
from datetime import datetime
from typing import Dict, List, Any
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

print("Libraries imported successfully!")

In [ ]:
# Connect to Redis
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

# Test connection
try:
    r.ping()
    print("✅ Redis connection successful!")
    print(f"Redis server version: {r.info()['redis_version']}")
except redis.ConnectionError:
    print("❌ Failed to connect to Redis. Please ensure Redis is running on localhost:6379")

## Part 1: Configure Redis for Advanced Monitoring

Configure Redis settings for monitoring slow queries and latency issues.

In [ ]:
# Configure slow query logging
r.config_set('slowlog-log-slower-than', 10000)  # Log queries slower than 10ms
r.config_set('slowlog-max-len', 128)  # Keep last 128 slow queries

# Configure latency monitoring
r.config_set('latency-monitor-threshold', 100)  # Monitor latencies > 100ms

# Display current configuration
slowlog_config = r.config_get('slowlog*')
latency_config = r.config_get('latency*')

print("📊 Redis Monitoring Configuration:")
print("\nSlow Query Settings:")
for key, value in slowlog_config.items():
    print(f"  {key}: {value}")

print("\nLatency Monitoring Settings:")
for key, value in latency_config.items():
    print(f"  {key}: {value}")

## Part 2: System Metrics Collection

Create a comprehensive monitoring system for insurance operations.

In [ ]:
class InsuranceMonitor:
    """Advanced monitoring system for insurance Redis operations"""
    
    def __init__(self, redis_client):
        self.redis = redis_client
        self.start_time = time.time()
    
    def get_system_metrics(self) -> Dict[str, Any]:
        """Collect comprehensive system metrics"""
        info = self.redis.info()
        
        metrics = {
            'Server': {
                'version': info.get('redis_version', 'N/A'),
                'uptime_days': info.get('uptime_in_days', 0),
                'config_file': info.get('config_file', 'N/A')
            },
            'Clients': {
                'connected': info.get('connected_clients', 0),
                'blocked': info.get('blocked_clients', 0),
                'max_clients': info.get('maxclients', 'N/A')
            },
            'Memory': {
                'used_memory': info.get('used_memory_human', 'N/A'),
                'peak_memory': info.get('used_memory_peak_human', 'N/A'),
                'memory_fragmentation': info.get('mem_fragmentation_ratio', 'N/A')
            },
            'Performance': {
                'ops_per_sec': info.get('instantaneous_ops_per_sec', 0),
                'total_commands': info.get('total_commands_processed', 0),
                'keyspace_hits': info.get('keyspace_hits', 0),
                'keyspace_misses': info.get('keyspace_misses', 0)
            }
        }
        
        # Calculate hit rate
        hits = metrics['Performance']['keyspace_hits']
        misses = metrics['Performance']['keyspace_misses']
        if hits + misses > 0:
            metrics['Performance']['hit_rate'] = f"{(hits / (hits + misses)) * 100:.2f}%"
        else:
            metrics['Performance']['hit_rate'] = "N/A"
        
        return metrics
    
    def get_insurance_key_stats(self) -> Dict[str, int]:
        """Get statistics for insurance-specific keys"""
        patterns = {
            'total_keys': self.redis.dbsize(),
            'policies': len(list(self.redis.scan_iter('policy:*'))),
            'customers': len(list(self.redis.scan_iter('customer:*'))),
            'claims': len(list(self.redis.scan_iter('claim:*'))),
            'quotes': len(list(self.redis.scan_iter('quote:*'))),
            'agents': len(list(self.redis.scan_iter('agent:*')))
        }
        return patterns

# Create monitor instance and display metrics
monitor = InsuranceMonitor(r)
metrics = monitor.get_system_metrics()

print("📊 System Metrics Dashboard")
print("=" * 50)
for category, values in metrics.items():
    print(f"\n{category}:")
    for key, value in values.items():
        print(f"  {key:<20} {value}")

In [ ]:
# Get insurance-specific key statistics
key_stats = monitor.get_insurance_key_stats()

print("🔑 Insurance Key Statistics")
print("=" * 50)
for key_type, count in key_stats.items():
    print(f"  {key_type:<15} {count:>10,} keys")

# Create visualization
if sum(key_stats.values()) > 0:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Bar chart
    categories = list(key_stats.keys())[1:]  # Exclude total_keys
    values = [key_stats[k] for k in categories]
    ax1.bar(categories, values, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd'])
    ax1.set_ylabel('Number of Keys')
    ax1.set_title('Insurance Data Distribution')
    ax1.tick_params(axis='x', rotation=45)
    
    # Pie chart
    non_zero = {k: v for k, v in key_stats.items() if v > 0 and k != 'total_keys'}
    if non_zero:
        ax2.pie(non_zero.values(), labels=non_zero.keys(), autopct='%1.1f%%', startangle=90)
        ax2.set_title('Key Distribution by Type')
    
    plt.tight_layout()
    plt.show()

## Part 3: Performance Benchmarking

Benchmark typical insurance operations to identify performance characteristics.

In [ ]:
class PerformanceBenchmark:
    """Benchmark insurance operations"""
    
    def __init__(self, redis_client):
        self.redis = redis_client
        self.results = []
    
    def benchmark_operation(self, name: str, operation, iterations: int = 1000):
        """Benchmark a specific operation"""
        print(f"⚡ Benchmarking {name} ({iterations} iterations)...")
        
        start_time = time.time()
        for i in range(iterations):
            operation(i)
        duration = time.time() - start_time
        
        ops_per_sec = iterations / duration
        avg_latency_ms = (duration / iterations) * 1000
        
        result = {
            'operation': name,
            'iterations': iterations,
            'duration_sec': round(duration, 3),
            'ops_per_sec': round(ops_per_sec, 0),
            'avg_latency_ms': round(avg_latency_ms, 3)
        }
        
        self.results.append(result)
        print(f"  ✓ {ops_per_sec:,.0f} ops/sec, {avg_latency_ms:.3f}ms avg latency")
        return result
    
    def run_insurance_benchmarks(self):
        """Run comprehensive insurance operation benchmarks"""
        
        # Policy lookup
        self.benchmark_operation(
            'Policy Lookup',
            lambda i: self.redis.get(f'policy:AUTO-{100000 + (i % 100)}')
        )
        
        # Customer batch retrieval
        self.benchmark_operation(
            'Customer Batch Get',
            lambda i: self.redis.mget([f'customer:CUST{j:04d}' for j in range(i % 10, (i % 10) + 5)]),
            iterations=500
        )
        
        # Claims queue operations
        self.benchmark_operation(
            'Claims Queue',
            lambda i: [self.redis.lpush('claims:benchmark', f'CLM-{i}'), 
                      self.redis.rpop('claims:benchmark')]
        )
        
        # Premium calculations
        self.benchmark_operation(
            'Premium Updates',
            lambda i: [self.redis.set(f'premium:test:{i}', 1000),
                      self.redis.incrby(f'premium:test:{i}', 150),
                      self.redis.delete(f'premium:test:{i}')]
        )
        
        # Quote generation with TTL
        self.benchmark_operation(
            'Quote Generation',
            lambda i: self.redis.setex(f'quote:BENCH:{i}', 300, json.dumps({
                'quote_id': f'Q{i:06d}',
                'premium': 1200 + (i % 500),
                'coverage': 'comprehensive'
            }))
        )
        
        return pd.DataFrame(self.results)

# Run benchmarks
benchmark = PerformanceBenchmark(r)
results_df = benchmark.run_insurance_benchmarks()

print("\n📊 Benchmark Results Summary")
print("=" * 70)
print(results_df.to_string(index=False))

In [ ]:
# Visualize benchmark results
if len(results_df) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Operations per second
    ax1 = axes[0, 0]
    colors = plt.cm.viridis(range(len(results_df)))
    ax1.bar(results_df['operation'], results_df['ops_per_sec'], color=colors)
    ax1.set_ylabel('Operations/Second')
    ax1.set_title('Throughput Comparison')
    ax1.tick_params(axis='x', rotation=45)
    
    # Average latency
    ax2 = axes[0, 1]
    ax2.bar(results_df['operation'], results_df['avg_latency_ms'], color=colors)
    ax2.set_ylabel('Latency (ms)')
    ax2.set_title('Average Latency per Operation')
    ax2.tick_params(axis='x', rotation=45)
    
    # Duration comparison
    ax3 = axes[1, 0]
    ax3.bar(results_df['operation'], results_df['duration_sec'], color=colors)
    ax3.set_ylabel('Duration (seconds)')
    ax3.set_title('Total Benchmark Duration')
    ax3.tick_params(axis='x', rotation=45)
    
    # Performance comparison (normalized)
    ax4 = axes[1, 1]
    max_ops = results_df['ops_per_sec'].max()
    normalized = (results_df['ops_per_sec'] / max_ops) * 100
    ax4.barh(results_df['operation'], normalized, color=colors)
    ax4.set_xlabel('Performance (% of max)')
    ax4.set_title('Relative Performance')
    
    plt.tight_layout()
    plt.show()

## Part 4: Memory Analysis

Analyze memory usage patterns for insurance data.

In [ ]:
def analyze_memory_usage(redis_client):
    """Analyze memory usage for different key types"""
    
    print("💾 Memory Usage Analysis")
    print("=" * 50)
    
    # Sample keys for memory analysis
    sample_keys = [
        ('policy:AUTO-100001', 'Auto Policy'),
        ('customer:CUST001', 'Customer Profile'),
        ('claim:CLM-2024-001', 'Claim Record'),
        ('quote:AUTO:Q100001', 'Quote Data'),
        ('agent:AG001', 'Agent Profile')
    ]
    
    memory_data = []
    
    for key, description in sample_keys:
        if redis_client.exists(key):
            try:
                # Get memory usage (Redis 4.0+)
                usage = redis_client.memory_usage(key)
                key_type = redis_client.type(key)
                ttl = redis_client.ttl(key)
                
                memory_data.append({
                    'key': key,
                    'description': description,
                    'type': key_type,
                    'memory_bytes': usage,
                    'ttl': 'No expiry' if ttl == -1 else f'{ttl}s'
                })
                
                print(f"  {description:<20} {usage:>8} bytes (TTL: {ttl})")
            except:
                print(f"  {description:<20} Memory usage not available")
        else:
            print(f"  {description:<20} Key not found")
    
    # Get overall memory stats
    try:
        mem_stats = redis_client.memory_stats()
        print(f"\n📊 Overall Memory Statistics:")
        print(f"  Peak allocated: {mem_stats.get('peak.allocated', 'N/A')}")
        print(f"  Total allocated: {mem_stats.get('total.allocated', 'N/A')}")
        print(f"  Fragmentation ratio: {mem_stats.get('fragmentation', 'N/A')}")
    except:
        info = redis_client.info('memory')
        print(f"\n📊 Memory Info:")
        print(f"  Used memory: {info.get('used_memory_human', 'N/A')}")
        print(f"  Peak memory: {info.get('used_memory_peak_human', 'N/A')}")
        print(f"  Fragmentation: {info.get('mem_fragmentation_ratio', 'N/A')}")
    
    return memory_data

# Analyze memory
memory_analysis = analyze_memory_usage(r)

## Part 5: Slow Query Analysis

Analyze slow queries to identify performance bottlenecks.

In [ ]:
def analyze_slow_queries(redis_client, num_queries=10):
    """Analyze recent slow queries"""
    
    print(f"🐌 Analyzing Top {num_queries} Slow Queries")
    print("=" * 70)
    
    slow_queries = redis_client.slowlog_get(num_queries)
    
    if not slow_queries:
        print("No slow queries found. Great performance!")
        return []
    
    query_data = []
    
    for i, query in enumerate(slow_queries, 1):
        command = ' '.join(query['command'][:10])  # First 10 parts
        if len(query['command']) > 10:
            command += '...'
        
        duration_ms = query['duration'] / 1000
        timestamp = datetime.fromtimestamp(query['start_time']).strftime('%Y-%m-%d %H:%M:%S')
        
        query_data.append({
            'rank': i,
            'command': command[:50],  # Truncate long commands
            'duration_ms': duration_ms,
            'timestamp': timestamp
        })
        
        print(f"\n  #{i} - {duration_ms:.2f}ms")
        print(f"     Command: {command[:60]}")
        print(f"     Time: {timestamp}")
    
    return query_data

# Analyze slow queries
slow_query_data = analyze_slow_queries(r)

if slow_query_data:
    df_slow = pd.DataFrame(slow_query_data)
    print("\n📊 Slow Query Summary:")
    print(df_slow[['rank', 'duration_ms', 'command']].to_string(index=False))

## Part 6: Real-time Claims Monitoring

Monitor claims processing in real-time.

In [ ]:
class ClaimsMonitor:
    """Real-time claims monitoring system"""
    
    def __init__(self, redis_client):
        self.redis = redis_client
        self.reset_metrics()
    
    def reset_metrics(self):
        """Reset daily metrics"""
        self.redis.set('metrics:claims:submitted:today', 0)
        self.redis.set('metrics:claims:processed:today', 0)
        self.redis.set('metrics:claims:rejected:today', 0)
        self.redis.set('metrics:claims:pending:today', 0)
    
    def simulate_claims(self, num_claims=30):
        """Simulate claims processing"""
        import random
        
        print(f"\n🚀 Simulating {num_claims} insurance claims...")
        
        for i in range(num_claims):
            claim_id = f'CLM-2024-{i:06d}'
            
            # Submit claim
            self.redis.lpush('claims:pending', claim_id)
            self.redis.incr('metrics:claims:submitted:today')
            
            # Process with some delay
            time.sleep(0.05)  # Simulate processing time
            
            # Process claim (70% approved, 30% rejected)
            claim = self.redis.rpop('claims:pending')
            if claim:
                if random.random() < 0.7:
                    self.redis.lpush('claims:approved', claim)
                    self.redis.incr('metrics:claims:processed:today')
                else:
                    self.redis.lpush('claims:rejected', claim)
                    self.redis.incr('metrics:claims:rejected:today')
            
            # Display progress every 10 claims
            if (i + 1) % 10 == 0:
                self.display_metrics()
        
        print("\n✅ Simulation complete!")
        self.display_final_metrics()
    
    def display_metrics(self):
        """Display current metrics"""
        submitted = int(self.redis.get('metrics:claims:submitted:today') or 0)
        processed = int(self.redis.get('metrics:claims:processed:today') or 0)
        rejected = int(self.redis.get('metrics:claims:rejected:today') or 0)
        pending = self.redis.llen('claims:pending')
        
        print(f"  📊 Claims - Submitted: {submitted}, Processed: {processed}, Rejected: {rejected}, Pending: {pending}")
    
    def display_final_metrics(self):
        """Display final metrics with visualization"""
        submitted = int(self.redis.get('metrics:claims:submitted:today') or 0)
        processed = int(self.redis.get('metrics:claims:processed:today') or 0)
        rejected = int(self.redis.get('metrics:claims:rejected:today') or 0)
        pending = self.redis.llen('claims:pending')
        
        print("\n📈 Final Claims Metrics:")
        print("=" * 40)
        print(f"  Total Submitted: {submitted}")
        print(f"  Approved: {processed} ({processed/submitted*100:.1f}%)")
        print(f"  Rejected: {rejected} ({rejected/submitted*100:.1f}%)")
        print(f"  Still Pending: {pending}")
        
        # Create visualization
        if submitted > 0:
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
            
            # Status distribution
            statuses = ['Approved', 'Rejected', 'Pending']
            counts = [processed, rejected, pending]
            colors = ['#2ecc71', '#e74c3c', '#f39c12']
            ax1.bar(statuses, counts, color=colors)
            ax1.set_ylabel('Number of Claims')
            ax1.set_title('Claims Status Distribution')
            
            # Pie chart
            ax2.pie(counts, labels=statuses, colors=colors, autopct='%1.1f%%', startangle=90)
            ax2.set_title('Claims Processing Breakdown')
            
            plt.tight_layout()
            plt.show()

# Create claims monitor and run simulation
claims_monitor = ClaimsMonitor(r)
claims_monitor.simulate_claims(30)

## Part 7: Key Pattern Analysis

Analyze key patterns and identify potential optimization opportunities.

In [ ]:
def analyze_key_patterns(redis_client):
    """Analyze key naming patterns and distribution"""
    
    print("🔍 Key Pattern Analysis")
    print("=" * 50)
    
    patterns = {
        'policy:*': 'Policies',
        'customer:*': 'Customers',
        'claim:*': 'Claims',
        'quote:*': 'Quotes',
        'agent:*': 'Agents',
        'metrics:*': 'Metrics',
        'premium:*': 'Premiums',
        'claims:*': 'Claim Queues'
    }
    
    pattern_counts = {}
    
    for pattern, description in patterns.items():
        count = len(list(redis_client.scan_iter(pattern, count=100)))
        pattern_counts[description] = count
        print(f"  {description:<15} {count:>8} keys")
    
    total_keys = redis_client.dbsize()
    identified_keys = sum(pattern_counts.values())
    unidentified = total_keys - identified_keys
    
    print(f"\n  {'Total Keys':<15} {total_keys:>8}")
    print(f"  {'Identified':<15} {identified_keys:>8} ({identified_keys/total_keys*100:.1f}%)")
    print(f"  {'Unidentified':<15} {unidentified:>8} ({unidentified/total_keys*100:.1f}%)")
    
    # Find largest keys
    print("\n📏 Top 5 Largest Keys:")
    key_sizes = []
    
    for key in redis_client.scan_iter(count=100):
        try:
            size = redis_client.memory_usage(key)
            if size:
                key_sizes.append((key, size))
        except:
            pass
    
    key_sizes.sort(key=lambda x: x[1], reverse=True)
    
    for i, (key, size) in enumerate(key_sizes[:5], 1):
        print(f"  {i}. {key[:40]:<40} {size:>10,} bytes")
    
    return pattern_counts

# Analyze patterns
pattern_analysis = analyze_key_patterns(r)

## Lab 5 Summary and Conclusions

### ✅ Completed Tasks

In this lab, you have successfully:

1. **Configured Redis** for advanced monitoring (slow queries, latency)
2. **Collected system metrics** including memory, performance, and client stats
3. **Benchmarked insurance operations** to understand performance characteristics
4. **Analyzed memory usage** patterns for different data types
5. **Monitored slow queries** to identify bottlenecks
6. **Simulated real-time claims processing** with metrics tracking
7. **Analyzed key patterns** to understand data distribution

### 📊 Key Performance Insights

Based on the benchmarks and analysis:
- Simple GET operations provide the highest throughput
- Batch operations (MGET) are more efficient than multiple individual operations
- Queue operations (LPUSH/RPOP) are suitable for high-volume claims processing
- TTL-based operations (SETEX) add minimal overhead for quote management

### 🎯 Best Practices for Production

1. **Monitor continuously** - Set up dashboards for real-time monitoring
2. **Analyze slow queries regularly** - Identify and optimize bottlenecks
3. **Track memory usage** - Prevent memory issues before they occur
4. **Use appropriate data structures** - Choose the right Redis type for each use case
5. **Implement metrics collection** - Track business KPIs alongside technical metrics

### 🚀 Next Steps

- Implement automated alerting based on metrics thresholds
- Create custom dashboards for insurance operations
- Integrate monitoring with your application logging
- Set up regular performance benchmarking
- Explore Redis Sentinel for high availability monitoring

In [ ]:
# Final cleanup (optional)
print("🧹 Cleaning up test data...")

# Clean up benchmark and test keys
for key in r.scan_iter('premium:test:*'):
    r.delete(key)
for key in r.scan_iter('quote:BENCH:*'):
    r.delete(key)

# Clear test queues
r.delete('claims:benchmark')

print("✅ Lab 5 completed successfully!")
print("\n📚 You are now ready to monitor Redis in production environments!")